In [ ]:
# pip install import-ipynb
import pandas as pd
import numpy as np
import sqlite3
from sqlite3 import Error
from datetime import datetime

In [ ]:
import import_ipynb
from Create_Labels import labels_df
labels_df

In [ ]:
database = "transcripts1.db"
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
 
    return conn



In [ ]:
conn = create_connection(database)
c = conn.cursor()

In [ ]:
df = pd.read_sql_query('SELECT * FROM TRANSCRIPTS;', conn)
print(df.shape)
df.head()

In [ ]:
for i in range(df.shape[0]):
    df['DATE'][i] = df['DATE'][i].replace(".","")
    df['DATE'][i] = df['DATE'][i].replace(",","")


for i in range(df.shape[0]):
    if len(df['DATE'][i]) <20:
        try:
            df['DATE'][i] =  datetime.strptime(df['DATE'][i].strip(), '%b %d %Y')
        except:
            df['DATE'][i] =  datetime.strptime(df['DATE'][i].strip(), '%B %d %Y')
    else:
        try:
            df['DATE'][i] = df['DATE'][i].split("ending",1)[1].strip()
            df['DATE'][i] =  datetime.strptime(df['DATE'][i].strip(), '%b %d %Y')
        except:
            try:
                df['DATE'][i] =  datetime.strptime(df['DATE'][i].strip(), '%B %d %Y')
            except:
                pass

In [ ]:
df["DATE"] = pd.to_datetime(df["DATE"], errors = 'coerce')
df.dropna(how='any', inplace = True)
df['YEAR'] = df['DATE'].dt.year.astype(int).astype(str)
df['MONTH'] = df['DATE'].dt.month.astype(int).astype(str)
df['WEEK'] = df['DATE'].dt.week.astype(int).astype(str)
df['TRANSCRIPTS'][0]

In [ ]:
df['PRIMARY_KEY'] = df['TICKER'] + "_" + df['YEAR'] 
df.head()

In [ ]:
labels_df["DATE"] = pd.to_datetime(labels_df["date"], errors = 'coerce')
labels_df['YEAR'] = labels_df['DATE'].dt.year.astype(int).astype(str)
labels_df['MONTH'] = labels_df['DATE'].dt.month.astype(int).astype(str)
labels_df['PRIMARY_KEY'] = labels_df['tickers'] + "_" + labels_df['YEAR']
labels_df.head()
# labels_sorted  = labels_df.sort_values(by=['tickers','YEAR','MONTH'])
# labels_sorted["NEXT_MONTH"] = labels_sorted['MONTH'].shift(+1)
# labels_sorted['PRIMARY_KEY'] = labels_sorted['tickers'] + "_" + labels_sorted['YEAR'] + "_"+ labels_sorted['MONTH']+ "_"+ labels_sorted['NEXT_MONTH']
# labels_sorted.head()

In [ ]:
# df_sorted  = df.sort_values(by=['TICKER','YEAR','MONTH'])
# df_sorted["NEXT_MONTH"] = df_sorted['MONTH'].shift(+1)
# df_sorted['PRIMARY_KEY'] = df_sorted['TICKER'] + "_" + df_sorted['YEAR'] + "_"+ df_sorted['MONTH']+ "_"+ df_sorted['NEXT_MONTH']
# df_sorted.head(15)

In [ ]:
final_df = df.merge(labels_df, left_on='PRIMARY_KEY', right_on='PRIMARY_KEY', how = 'inner')
final_df.shape

In [ ]:
final_df['DATE_DIFF'] = final_df['DATE_y'] - final_df['DATE_x']
final_df = final_df[final_df['DATE_DIFF'] <= pd.Timedelta(90,'D')]
final_df = final_df[final_df['DATE_DIFF'] >= pd.Timedelta(10,'D')]

final_df['TRANSCRIPT_DATE'] = final_df['DATE_x']
final_df['RATINGS_DATE'] = final_df['DATE_y']
final_df['KEY'] = final_df['TICKER'].astype('str') + "_" +  final_df['RATINGS_DATE'].astype('str')
                                       
final_df = final_df[['TRANSCRIPTS','TICKER','TRANSCRIPT_DATE','RATINGS_DATE','ratings','DATE_DIFF','KEY']]


In [ ]:
print(final_df.shape[0])
(final_df['TICKER'].astype('str') + "_" + final_df['RATINGS_DATE'].astype('str')).nunique()

In [ ]:
final_df[final_df.duplicated(['KEY'])].sort_values(by='KEY')

In [ ]:
# df[df['TICKER'] == 'AAPL']
# labels_df[labels_df['tickers'] == 'AAPL']
print(final_df.shape)
final_df.sort_values(by=['TICKER','RATINGS_DATE'],inplace = True)
final_df.drop_duplicates(subset ="KEY", 
                     keep = "first", inplace = True)
final_df.shape